In [1]:
# Importar librerías

import pandas as pd
import matplotlib as mlt
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Para que las gráficas se impriman no en el notebook, sino en una nueva pestaña
%matplotlib qt

In [3]:
# Coordenadas de mundo
nm = 1e-9
um = 1e-6
mm = 1e-3
cm = 1e-2

# Longitud de onda
w_length = 600*nm

#
Lu = 4.2
Lv = 3.18


In [4]:
# Función para graficar complejos
def fun_ploteo_complejo(mat, indicador, escala, mapa_color = 'gray'):
    """
    mat  es una matriz compleja a plotear
    indicador = I para intensidad, A para amplitud, P para fase
    escala = 1 para nada, 0 para logarítmica
    """

    if(indicador =="I"):
        mat = np.abs(mat)**2
    elif(indicador == "A"):
        mat = np.abs(mat)
    elif(indicador == "P"):
        mat = np.angle(mat)

    if (escala == 1):
        plt.figure()
        plt.imshow(mat, cmap = mapa_color)
        plt.colorbar()
        plt.show()
    else:    
        plt.figure()
        plt.imshow( np.log( mat + 0.000000001), cmap = mapa_color)
        plt.colorbar()
        plt.show()
#----------------------------------------------------------------------------------------------------------------
# Función para "Pading"
def fun_pad(campo, tipo = 'c'):
    '''
    fun_pad es una función que "padea" un campo (imagen) 2D con ceros tal que duplica su longitud y altura

    Solo recibe imágenes de 1 canal

    Variables de entrada:
     - campo: (numpy.ndarray) ; es el campo a padear de L_x y L_y dimensiones físicas reales
     - tipo: (str) ; c = pad cuadrado, de lado y lado con la dimensión máxima de campo
                     o = pad rectangualr teniendo en cuenta las dimensiones de campo

    Variables de salida:
     - campo_paded: (numpy.ndarray) ; es el campo padeado con ceros de dimensiones 2*L_x y 2*L_y 
    '''

    
    if tipo == 'c':
        dim_max = max(np.shape(campo))
        
        dimension_pad = np.ceil(np.array( [ int(2*dim_max-np.shape(campo)[0]) , int(2*dim_max-np.shape(campo)[1]) ] ) / 2)
        campo_paded = np.pad(campo, ( (int(dimension_pad[0]), int(dimension_pad[0])-1 ), (int(dimension_pad[1]), int(dimension_pad[1]) ) ), mode = 'constant', constant_values = (0,0) )
    

    else:
        # extraer las dimensiones del pad
        dimension_pad = np.array( [np.shape(campo)[0] , np.shape(campo)[1] ] ) / 2

    # padear el campo con ceros
        campo_paded = np.pad(campo, ( (int(dimension_pad[0]), int(dimension_pad[0]) ), (int(dimension_pad[1]), int(dimension_pad[1])) ), mode = 'constant', constant_values = (0,0) )

    return (campo_paded)

#----------------------------------------------------------------------------------------------------------------
# Función calculadora del z crítico
def fun_z_c(M, dx, w_length):
    '''
    fun_z_c es una función que calcula el la distancia de propagación crítica (z crítico) donde los métodos
    de propagación AS y Fresnel funcionan, dados un campo, tamaño de discretización y longitu de onda
    
    Variables de entrada
     - M: int ; número de muestreos
     - dx: float ; tamaño de discretización (tamaño del pixel)
     - w_length: float ; longitud de onda

    Variables de salida
     - z_c: float ; z crítico
    '''
    z_c = M*dx**2/w_length

    return z_c

#----------------------------------------------------------------------------------------------------------------
# Función para crear máscaras cuadradas y circulares centradas en la imagen
def fun_mask(dim, L_R, tipo = 1, forma ='circ'):
    '''
    fun_mask es una función que crea y grafica máscaras ciruclares o cuadradas centradas en la imagen
    
    Variables de entrada
     - dim: int ; dimensión del campo
     - L_R: int ; tamaño de la máscara (radio o longitud)
     - tipo: int ; 0 para un obstaculo
                 ; 1 para una apertura
     - forma: str ; 'cir' para circulos
                    'rect' para cuadrados

    Variables de salida
     - mascara: matriz de dimensiones  dim x dim con una mascara 
    '''

    if tipo == 0:
        mascara = np.ones(np.array([dim,dim]), dtype = "uint8") # uint8 tipo de la variable de la matriz (enteros de 8 bit)
    else:
        mascara = np.zeros(np.array([dim,dim]), dtype = "uint8") # uint8 tipo de la variable de la matriz (enteros de 8 bit)

    # Como mi imagen no es cuadrada, entonces necesito dos coordenadas
    # para el centro
    centro1 = int(mascara.shape[0]/2)
    centro2 = int(mascara.shape[1]/2)

    if forma == 'circ':
        #Graficar la máscara circular 
        cv2.circle(mascara, (centro1, centro2), L_R, tipo, -1)
        plt.figure()
        plt.imshow(mascara)
        plt.colorbar() # Para mostrar barra de colores
        plt.show()

    elif forma == 'rect':
        inicio = (int(centro1-L_R/2),int(centro2-L_R/2))
        fin = (int(centro1+L_R/2),int(centro2+L_R/2))
        cv2.rectangle(mascara, inicio, fin,  tipo, -1)
        plt.figure()
        plt.imshow(mascara)
        plt.colorbar() # Para mostrar barra de colores
        plt.show()
    
    return mascara

def fft(campo):
    fft_campo = np.fft.fftshift(np.fft.fftn(campo))

    return fft_campo

def ifft(campo):
    fft_campo = np.fft.fftshift(np.fft.ifftn(campo))
    return fft_campo


# Lectura de imagen 

In [5]:
# Lectura de la fotografía
gala = cv2.imread('gala.jpg')              # Lecutra del .csv como un objeto np.array
# Pasar de 3 canales a 1 (monocromático)
gala_gray = cv2.cvtColor(gala, cv2.COLOR_BGR2GRAY)

# # Padear la figura de manera cuadrada
# gala_paded = fun_pad(gala_gray)

# # Imprimir imagen padeada
# plt.imshow(gala_paded, cmap ='gray')

print("dimensión de la imagen original es: ", np.shape(gala))
# print("dimensión de la imagen padeada es:", np.shape(gala_paded))

ft_gala = fft(gala)
# ft_gala = fft(gala_paded)

plt.figure()
plt.imshow(gala_gray, cmap = 'gray')
plt.colorbar()
plt.show()

dimensión de la imagen original es:  (2835, 2182, 3)


# Funciones para simulación

In [6]:
#----------------------------------------------------------------------------------------------------------------
# Función para crear los deltas a una distancia z
def deltas(z_o, Lu, Lv, w_length = 600*nm ,z_i = 22*mm):
    """
    Función que genera los deltas del espacio de frecuencias a una distancia zo de la lente, este objeto tendrá
    dimensiones Lu, Lc, con espectro visible w_length,  a una distancia de la lente z_i

    """

    dfu = z_o * w_length/ (z_i* Lu )
    dfv = z_o * w_length/(z_i* Lv )

    return np.array([dfu,dfv])
#----------------------------------------------------------------------------------------------------------------
# Función para calcular la frecuencia de corte obtenida por el análisis de sistemas generalizados
def f_critico (z_o, w_length, r = 3*mm):
    """
    Función que calcula la frecuencia de corte de un ojo humano, recibe la distancia del objeto z_o, la longitud
    de onda del espectro visible w_length y el radio de la pupila r
    """
    f_c = r/(w_length*z_o)

    return f_c
#----------------------------------------------------------------------------------------------------------------
# Función para calcular la función de la pupila escalada
def pupila(campo, z_o, Lu, Lv, w_length, r = 3*mm, z_i = 22*mm):
    """
    Función que genera una máscara simulando el efecto de la pupila del ojo humano, las frecuencias permitidas
    serán aquellas que sean menor al valor de frecuencia de corte.
    Recibe el campo en el plano objeto ubicado a una distancia z_o, con dimensiones Lu, Lv, con longitud de onda
    del espectro visible w_length, el radio de la pupila r y la distancia a la retina z_i
    """
    N,M = np.shape(campo)

    dfu, dfv = deltas(z_o, Lu, Lv, z_i)

    p = np.arange(-int(np.floor(M/2) ), int(np.ceil(M/2) ), 1)
    q = np.arange(-int(np.floor(N/2) ), int(np.ceil(N/2) ), 1)
  
    P, Q = np.meshgrid(p, q)

    Fu = P*dfu
    Fv = Q*dfv

    f_c = f_critico(z_o, w_length, r)

    # generar el filtro pasa-bajas debido a la pupila
    circulo = np.array(Fu**2 + Fv**2 <= f_c**2, dtype = bool)


    plt.figure()
    plt.imshow(circulo)
    plt.colorbar()
    plt.show()

    return circulo

#----------------------------------------------------------------------------------------------------------------
# Función que simula el efecto de la formación de una imagen en la retina del ojo humano
def U_i(U_o, z_o, Lu, Lv, w_length = 600*nm, r = 3*mm,z_i =22*mm, ):
    """
    Esta función simula el efecto de formación de imágenes del ojo humano de un objeto con campo U_o
    a una distancia z_o, con dimensiones Lu,Lv, longitu de onda longitud de onda
    del espectro visible w_length, el radio de la pupila r y la distancia a la retina z_i
    """
    # Obener espectro del campo
    fft_U_o = fft(U_o)
    
    # Crear la pupila del ojo específica a esa distancia para ese objeto
    pupila_ojo = pupila(U_o, z_o, Lu, Lv, w_length, r , z_i)

    # "Convolucionar"
        # filtrar las frecuencias 
    fft_U_i = fft_U_o * pupila_ojo
        # Obtener el campo real
    U_i = np.fft.fftshift(ifft(fft_U_i))

    # Graficar
    fun_ploteo_complejo(U_i, "I", 1)

    return U_i


# Simulación 

In [7]:
# Simular el efecto de un objeto a una distancia z_o
simulacion = U_i(gala_gray, 17, Lu, Lv)

In [46]:
def deltas(z_o, Lu, Lv, w_length = 600*nm ,z_i = 22*mm):
    dfu = z_o * w_length/ (z_i* Lu )
    dfv = z_o * w_length/(z_i* Lv )

    return np.array([dfu,dfv])

def f_critico (z_o, w_length, r = 3*mm):
    f_c = r/(w_length*z_o)

    return f_c

def pupila(campo, z_o, Lu, Lv, w_length, r = 3*mm, z_i = 22*mm):
    # Esta es otra función de creación de máscaras 
    N,M = np.shape(campo)

    dfu, dfv = deltas(z_o, Lu, Lv, z_i)

    p = np.arange(-int(np.floor(M/2) ), int(np.ceil(M/2) ), 1)
    q = np.arange(-int(np.floor(N/2) ), int(np.ceil(N/2) ), 1)
  
    P, Q = np.meshgrid(p, q)

    Fu = P*dfu
    Fv = Q*dfv

    f_c = f_critico(z_o, w_length, r)

    circulo = np.array(Fu**2 + Fv**2 <= f_c**2, dtype = bool)


    plt.figure()
    plt.imshow(circulo)
    plt.colorbar()
    plt.show()

    return circulo
    

In [51]:
print("f critico: ", f_critico(10, w_length) )
print("deltas: ", deltas(10, 4.2, 3.18))


pupila_filtro = pupila(gala_gray, 45, 4.2, 3.18, w_length)

print("dimensiones de la pupila: ",pupila_filtro.shape)

f critico:  499.99999999999994
deltas:  [6.49350649e-05 8.57632933e-05]
dimensiones de la pupila:  (2835, 2182)


In [52]:
fun_ploteo_complejo(np.fft.fftshift(ifft(fft(gala_gray)*pupila_filtro)), "A", 0)

In [12]:
fun_ploteo_complejo(np.fft.fftshift(ifft(ft_gala*pupila_ojo)), "I", 1)

In [ ]:
 https://wa.me/573015882452

In [ ]:
 https://wa.me/573505077894

In [47]:
2*20*np.tan(np.radians(2.5/60) )

0.02908882599444451

In [62]:
fft_gala = fft(gala_paded)

In [64]:
fun_ploteo_complejo(fft_gala, "I",0)